<a href="https://colab.research.google.com/github/adwoaelegba/hybridsummodel/blob/main/Final_Koios_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Koios Model
This model is a mix of extractive and abstractive summarization. Extractive summarization is meant to extract the most important phrases from the main text. The important sentences are then passed through the abstractive model (built using BART) to generate a user-friendly summary

# Installing libraries

In [ ]:
pip install torch transformers datasets accelerate peft bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install torch_xla[tpu] -f https://storage.googleapis.com/libtpu-releases/index.html


Looking in links: https://storage.googleapis.com/libtpu-releases/index.html


In [ ]:
pip install bs4

In [ ]:
pip install sentence_transformers

In [ ]:
pip install nltk

# Extraction Function
This function is meant to scrape the HTML from the site and feed it into the model

In [ ]:
def extraction_function(html):
    from bs4 import BeautifulSoup

    with open(html, "r", encoding="utf-8") as file:
        content = file.read()
        soup = BeautifulSoup(content, "html.parser")

    sections = {}
    current_heading = None
    current_text = []  # Initialize

    for elem in soup.find_all(["h1", "h2", "h3", "p"]):

        if elem.name in ["h1", "h2", "h3"]:
            if current_heading:
                sections[current_heading] = " ".join(current_text).strip()
            current_heading = elem.get_text().strip()
            current_text = []
        else:
            current_text.append(elem.get_text().strip())

    if current_heading and current_text:
        sections[current_heading] = " ".join(current_text).strip()


    return sections

Testing the extraction function

In [ ]:
html_file= "/content/drive/MyDrive/corpus/htmltest/www8_hp.html"
policy=extraction_function(html_file)
for section, text in policy.items():
    print(f"## {section} ###\n{text}\n")

## (No Subtitle) ###
HP respects your privacy

HP and its subsidiaries respect your privacy. This Privacy Statement informs you of our privacy practices and of the choices you can make about the way information about you, including from your online activity is collected and how that information is used by HP. This statement is readily available on our HP.com home page and at the bottom of every HP web page.

HP was a founding sponsor of the Council of Better Business Bureau's Online Privacy Program and is a BBB Accredited Business. HP privacy practices are consistent with the BBB Code of Business Practices and we proudly display the BBB seal.

Privacy BBB OnLinenon-HP site

In the development of HP's privacy policies and standards, we respect and take into account the major principles and frameworks around the world, including the OECD Guidelines on the Protection of Privacy and Transborder Flows, EU Directive 95/46/EC, APEC Privacy Framework, and the Madrid Resolution on International

# Extractive Summarization
This uses TD-IF to select the sentences with the highest importance. The measurement for importance is based on the privacy terms

In [ ]:
import torch
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sent_model=SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Key privacy terms to prioritize
privacy_keywords={
    "privacy","data","personal","information","collection","third-party","cookies","retention",
    "cookies","sharing","consent","security","policy","tracking", "advertising","partners",
    "compliance","regulation","opt-out", "security"
}

In [ ]:
#Legal phrases that should be kept
legal_phrases={
    "third-party sharing", "right to be forgotten","legitimate interest","data retention",
    "opt-out option","cookie tracking"
}

In [ ]:
#Stopwords to remove to improve TD-IDF scores but keep the ones that are needed for legal phrases (i.e to be)
stopwords=list(set(nltk.corpus.stopwords.words("english")).union({"shall","hereby","thereof","whereas"}) - set("to be".split()))

In [ ]:
#Function to preprocess text, keeping legal phrases. Changed text to a single token
def preprocess_text(text):
    for phrase in legal_phrases:
        text=text.replace(phrase,phrase.replace(" ","_"))
    return text


In [ ]:
def td_extract_summary(text, boost_factor=2.0):
    text = preprocess_text(text)
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)

    if total_sentences < 5:
        return text
    if total_sentences <= 10:
        num_sentences = max(5, int(total_sentences * 0.80))
    elif total_sentences <= 20:
        num_sentences = max(9, int(total_sentences * 0.5))
    else:
        num_sentences = max(11, int(total_sentences * 0.4))

    # TF-IDF
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    tdidf_matrix = vectorizer.fit_transform(sentences)
    tdidf_scores = np.asarray(tdidf_matrix.mean(axis=0)).flatten()
    word_to_tdidf = dict(zip(vectorizer.get_feature_names_out(), tdidf_scores))

    sentence_weights = []
    for sentence in sentences:
        words = sentence.lower().split()
        weight = sum(word_to_tdidf.get(word, 0) for word in words)
        if any(word in privacy_keywords for word in words):
            weight *= boost_factor
        sentence_weights.append(weight)

    sentence_weights = np.array(sentence_weights)
    max_weight = np.max(sentence_weights) if np.max(sentence_weights) > 0 else 1
    sentence_weights = (sentence_weights / max_weight) * boost_factor

    # Encode sentences
    sentence_embeddings = sent_model.encode(sentences, convert_to_tensor=True)
    doc_embedding = torch.mean(sentence_embeddings, dim=0)

    # Compute similarities
    similarities = torch.nn.functional.cosine_similarity(sentence_embeddings, doc_embedding.unsqueeze(0))

    # Move sentence_weights to the same device as similarities
    sentence_weights_tensor = torch.tensor(sentence_weights, dtype=torch.float32, device=similarities.device)
    similarities = similarities * sentence_weights_tensor  # No more device mismatch!

    # Select top-ranked sentences
    top_indices = torch.topk(similarities, num_sentences).indices.cpu().numpy()  # Move to CPU for indexing

    # Maintain original order of selected sentences
    top_sentences = [sentences[i].replace("_", " ") for i in sorted(top_indices)]  # Restore legal phrases

    return " ".join(top_sentences)


# Abstractive Summarization

# PEGASUS

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = "google/pegasus-large"
ptokenizer = PegasusTokenizer.from_pretrained(model_name)
pmodel = PegasusForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

# Final Pipeline (TD-IDF+Pegasus)


In [ ]:
def sum_text(input_text):
    if not input_text or len(input_text.strip()) == 0:
        print("Warning: Empty input text for summarization.")
        return "No summary available."

    inputs = ptokenizer(input_text, truncation=True, max_length=1024, return_tensors="pt")

    summary_ids = pmodel.generate(
        **inputs,
        max_length=150,
        min_length=50,
        num_beams=3,  # Reduce beams to allow more variation
        length_penalty=1.5,  # Encourage better-structured sentences
        repetition_penalty=2.0,  # Prevent repeating words
        no_repeat_ngram_size=3,  # Avoid repeating phrases
        temperature=0.9,  # Increases diversity (higher = more random)
        top_k=50,  # Sample from the top 50 words instead of picking the best
        top_p=0.85,  # Allows some randomness while keeping coherence
        do_sample=True  # Enables sampling (more natural summaries)
    )

    summary = ptokenizer.decode(summary_ids[0], skip_special_tokens=True)

    if not summary or len(summary.strip()) == 0:
        print("Warning: Model returned an empty summary.")
        return "No summary available."

    return summary


In [ ]:
#works
url = "/content/drive/MyDrive/corpus/htmltrain/www_apple.html"
extracted_summary = extraction_function(url)

if not extracted_summary:
    print("Error: No text extracted from the document.")
    exit()

final_summary = {}

for heading, text in extracted_summary.items():
    extractive_mod_summary = td_extract_summary(text)

    if not isinstance(extractive_mod_summary, str):
        print(f"Error: Extractive summary is {type(extractive_mod_summary)} instead of str.")
        continue

    # Ensure input is properly formatted
    cleaned_text = extractive_mod_summary.replace("\n", " ").strip()

    # Use the working abstractive function
    abstractive_mod_summary = sum_text(cleaned_text)

    final_summary[heading] = abstractive_mod_summary

# Print the final summaries
for heading, summary in final_summary.items():
    print(f"\n **{heading}**\n{summary}")



 **(No Subtitle)**
Privacy Policy Your privacy is important to Apple.So we've developed a Privacy Policy that covers how we collect, use, disclose, transfer, and store your information.The best way to protect your privacy is to be sure you are fully aware of what you are sharing with the world.

 **Collection and Use of Personal Information**
You may be asked to provide your personal information anytime you are in contact with Apple or an Apple affiliated company. Here are some examples of the types of personal information Apple may collect and how we may use it. This Privacy Policy does not apply to Apple and its affiliates, who may share this personal information with each other and use it consistent with this Privacy Policy.

 **What personal information we collect**
When you create an Apple ID, register your products, apply for commercial credit, purchase a product, download a software update, register for a class at an Apple Retail Store, or participate in an online survey, we ma

#Refining With GPT

In [ ]:
import openai

In [ ]:
def refined_text(text):
  client=openai.OpenAI(api_key=openai.api_key)

  response=client.chat.completions.create(
      model="gpt-4-turbo",
      messages=[{"role": "user", "content":f"Rewrite the folowing privact policy section using emojis an simple and engaging language:\n\n{text}"}],
      max_tokens=150
  )
  return response.choices[0].message.content

In [ ]:
#with refinement from GPT
#works
url = "/content/drive/MyDrive/corpus/htmltrain/www_apple.html"
extracted_summary = extraction_function(url)

if not extracted_summary:
    print("Error: No text extracted from the document.")
    exit()

final_summary = {}

for heading, text in extracted_summary.items():
    extractive_mod_summary = td_extract_summary(text)

    if not isinstance(extractive_mod_summary, str):
        print(f"Error: Extractive summary is {type(extractive_mod_summary)} instead of str.")
        continue

    # Ensure input is properly formatted
    cleaned_text = extractive_mod_summary.replace("\n", " ").strip()

    # Use the working abstractive function
    abstractive_mod_summary = sum_text(cleaned_text)

    refined_summary=refined_text(abstractive_mod_summary)

    final_summary[heading] = refined_summary

# Print the final summaries
for heading, summary in final_summary.items():
    print(f"\n **{heading}**\n{summary}")




 **(No Subtitle)**
🔒 Privacy Policy 🍏

Your privacy matters to us at Apple! 🌟 We have a Privacy Policy to explain how we handle your info - from collecting it to storing it. 📲💻

Please read it carefully and feel free to ask us anything! 🤔👍

 **Collection and Use of Personal Information**
👋 Every time you chat with Apple or our friends (affiliated companies), we might need some personal info from you. 📝 Here’s a quick look at what info we might ask for and how we use it:

✅ Collect info: We gather what’s needed to help make our services better for you.
✅ Use info: We use your details to improve your Apple experience.
✅ Share info: Sometimes, we share your details with our affiliates, but only to help us serve you better and always sticking to our privacy rules. 🔄

🔒 Your info is safe with us!

 **What personal information we collect**
🍏 When you make an Apple ID, buy stuff, sign up for classes, update software, or join surveys, we get your details! This includes your 📛, 🏠 address, 📞, 📧

# Flesch-Kincaid Test For Summary Readability

In [3]:
#installing the textstat library for Flesch and Flesch-Kincaid Scores
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.8 MB/s eta 0:00:00


In [4]:
import textstat

In [5]:
#testing section by section to identify if there are any difficult sections
summary_sections=[
    "Who's in charge of keeping your info safe when you use our website? Most of the time, it's ABL! But, if you're buying tickets, signing up for memberships, getting goodies from our store, or involved in any other cool stuff with us like our soccer schools, then it's Arsenal who keeps your data secure.",
    "Hey kiddos under 13! Wanna join in on fun contests and snag cool prizes? Make sure your parent or guardian signs you up! If you win, we’ll tell them first (to Mom or Dad!) to keep things super safe. Under 18 and won something? Awesome! We’ll need a thumbs-up from your parents before we share any of your personal stuff to celebrate your win!Remember, always play it safe and fun!",
    "Hey there! Just a heads up, we occasionally update our privacy and security policies.Make sure to swing by this page once in a while to stay in the loop, or you can hit up our Data Protection Officer to snag the latest copy! " ,
    "Hey there! Need to fix or update your info? It's easy! Just tweak your profile settings online or shoot a message to our Data Protection Officer (details below).Keep your details fresh and up-to-date to stay in the loop! And don’t forget, you can also find us hanging out on Facebook, Twitter, and Google+!",
    "Hey there! Just so you know: We might ask for a small fee (£10.00) to help cover our costs when you request updates or want to see your info.You can ask us anytime to fix your Personal Information to keep it accurate and fresh! You have the right to check out the details of the info we have about you. Keep it simple and stay informed!",
    "Sometimes, to get you the stuff you need, we send your info to other places where privacy laws might not be as strong as in the UK. But don't worry! We'll do our best to keep your information safe and sound, no matter where it goes!",
    "We might share your info with our trusted buddies - like sponsors, money helpers, and others we need to help get your orders to you, keep our systems running smooth, or follow the law.",
    "This includes your name, where you live, your phone number, when you were born, credit card details, where we should send bills, your email, and what you like to buy. We promise to use this info carefully and keep it safe, just like we said in our Privacy Policy.",
    "When you use our website or chat with us through other ways, if you share your info, it means you're cool with us using it based on our privacy rules.If you're not okay with it, just let us know!",
    "Your info is super safe on our secure server! Only a few of our team members have the key, and they can only peek when they really need to. Plus, we've got strong rules and steps to keep your data safe from any oopsies or nosy people! We only use your info carefully and wisely."
]

In [6]:
for idx, paragraph in enumerate(summary_sections):
  flesch_score=textstat.flesch_reading_ease(paragraph)
  flesch_k_grade=textstat.flesch_kincaid_grade(paragraph)
  print(f"Section {idx+1}:Flesch={flesch_score:.2f}, Flesch-Kincaid Grade={flesch_k_grade:.2f}")

Section 1:Flesch=70.13, Flesch-Kincaid Grade=8.00
Section 2:Flesch=87.92, Flesch-Kincaid Grade=3.20
Section 3:Flesch=58.11, Flesch-Kincaid Grade=10.50
Section 4:Flesch=66.44, Flesch-Kincaid Grade=7.30
Section 5:Flesch=83.46, Flesch-Kincaid Grade=4.90
Section 6:Flesch=81.33, Flesch-Kincaid Grade=5.70
Section 7:Flesch=54.90, Flesch-Kincaid Grade=13.80
Section 8:Flesch=71.99, Flesch-Kincaid Grade=9.30
Section 9:Flesch=77.06, Flesch-Kincaid Grade=7.40
Section 10:Flesch=74.39, Flesch-Kincaid Grade=6.30
